# Goals

I'd like to load in, process, and save the data so that I can just load it in for the figure plotting

# Imports

In [3]:
from pathlib import Path
import yaml
from tqdm import tqdm
import avoidome.uniprot as uniprot
from importlib import reload

In [4]:
data_dir = Path('../data')
fig_dir = Path('../figures')
uniprot_dir = data_dir / 'uniprot_downloads'
af_dir = data_dir / 'alphafold_downloads'

In [2]:
with open(data_dir / 'admet_names_curated.yml') as f:
    adme_names = yaml.safe_load(f)